In [1]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from io import StringIO
import os

In [2]:
# Leer los datos
df = pd.read_csv(f"car_prices.csv")

In [4]:
#Itero sobre columnas float para convertirlas en INT
columnas_iterar = ['condition','odometer','mmr','sellingprice']

for columna in columnas_iterar:
    # Verificar si hay valores nulos antes de llenarlos
    if df[columna].isnull().any():
        # Llenar los valores nulos con 0 y convertir la columna a tipo int
        df[columna] = df[columna].fillna(0).astype(int)
    else:
        # Si no hay valores nulos, simplemente convertir la columna a tipo int
        df[columna] = df[columna].astype(int)

In [5]:
# Convertir la columna 'saledate' de texto a fecha sin la parte de la hora
df['saledate'] = pd.to_datetime(df['saledate'], utc=True, errors='coerce').dt.date

df['saledate'].fillna(method='ffill', inplace=True)

/var/folders/l9/_6tgm3nj0n50mfclq6_lfbtc0000gn/T/ipykernel_37023/2713055797.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['saledate'] = pd.to_datetime(df['saledate'], utc=True, errors='coerce').dt.date
/var/folders/l9/_6tgm3nj0n50mfclq6_lfbtc0000gn/T/ipykernel_37023/2713055797.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['saledate'].fillna(method='ffill', inplace=True)
/

In [6]:
# Crea un diccionario con las nuevas asignaciones
correcciones = {
    "transmission": df["vin"],
    "vin": df["state"],
    "state": None,
}

# Filtra los registros donde 'transmission' es "Sedan"
df_sedan = df[(df["transmission"] == "Sedan") & (df["transmission"] == "sedan")]

# Actualiza las columnas en los registros filtrados
for col, val in correcciones.items():
    df_sedan[col] = val

# Actualiza la columna 'state' en el DataFrame original
df.loc[df_sedan.index, "state"] = df_sedan["state"]

In [7]:
load_dotenv(verbose=True)

# Usa las variables de entorno cargadas
DB_HOST = os.environ["DB_HOST"]
DB_DATABASE = os.environ["DB_DATABASE"]
DB_USER = os.environ["DB_USER"]
DB_PASSWORD = os.environ["DB_PASSWORD"]

In [8]:
# Conexión a la base de datos PostgreSQL
conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_DATABASE,
    user=DB_USER,
    password=DB_PASSWORD
)

In [9]:
# Crear un cursor
cur = conn.cursor()


# Convertir el DataFrame a formato CSV en memoria
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False, header=False)
csv_buffer.seek(0)

# Copiar los datos desde el CSV en memoria a la tabla en la base de datos
cur.copy_from(csv_buffer, "car_prices", sep=',')



In [10]:
# Consulta SQL para calcular la moda de 'model' agrupada por 'make', 'trim' y 'body'
query_model = """
    UPDATE car_prices
    SET model = subquery.mode_value
    FROM (
        SELECT make, trim, body, NULLIF(MODE() WITHIN GROUP (ORDER BY model), '') AS mode_value
        FROM car_prices
        GROUP BY make, trim, body
    ) AS subquery
    WHERE car_prices.make = subquery.make
    AND car_prices.trim = subquery.trim
    AND car_prices.body = subquery.body
    AND car_prices.model IS NOT NULL
"""

In [11]:
'''
query_transmission = """
    UPDATE car_prices
    SET transmission = subquery.mode_value
    FROM (
        SELECT model, NULLIF(MODE() WITHIN GROUP (ORDER BY transmission), '') AS mode_value
        FROM car_prices
        WHERE model IS NOT NULL
        GROUP BY model
    ) AS subquery
    WHERE car_prices.model = subquery.model
    AND car_prices.transmission IS NOT NULL
"""
'''

'\nquery_transmission = """\n    UPDATE car_prices\n    SET transmission = subquery.mode_value\n    FROM (\n        SELECT model, NULLIF(MODE() WITHIN GROUP (ORDER BY transmission), \'\') AS mode_value\n        FROM car_prices\n        WHERE model IS NOT NULL\n        GROUP BY model\n    ) AS subquery\n    WHERE car_prices.model = subquery.model\n    AND car_prices.transmission IS NOT NULL\n"""\n'

In [12]:
#Deletear registros que no voy a usar en mi analisis
delete_query = """
    DELETE FROM car_prices
    WHERE model IS NULL
    AND make = ''
    AND trim = '';
"""

In [13]:
# Defino consultas
consultas = [query_model] #,query_transmission]

# Ejecutar las consultas SQL
with conn.cursor() as cursor:
    for consulta in consultas:
        cursor.execute(consulta)
    # Confirmar la transacción después de ejecutar todas las consultas
    conn.commit()

    # Ejecutar la consulta SQL para eliminar registros
    cursor.execute(delete_query)
    # Confirmar la transacción después de eliminar registros
    conn.commit()

# Cargar los datos actualizados en un DataFrame de pandas
df_actualizado = pd.read_sql("SELECT * FROM car_prices", conn)

/var/folders/l9/_6tgm3nj0n50mfclq6_lfbtc0000gn/T/ipykernel_37023/2571184703.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_actualizado = pd.read_sql("SELECT * FROM car_prices", conn)
